In [ ]:
import os
import json
from collections import Counter

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

_ = nltk.download("stopwords", quiet=True)
_ = nltk.download("punkt", quiet=True)

In [ ]:
# Create the data/02_postprocessing direction if it does not exist
if not os.path.exists("data/02_postprocessing"):
    os.makedirs("data/02_postprocessing")
# Create the data/02_postprocessing/sample directory if it does not exist
if not os.path.exists("data/02_postprocessing/sample"):
    os.makedirs("data/02_postprocessing/sample")

In [ ]:
def silent_remove(filename):
    try:
        os.remove(filename)
    except OSError:
        pass

This notebook is designed so that it can be run on the sample data or the full data just by switching one variable.

In [ ]:
# Set this to True to run the script on the sample data
# Set this to False to run the script on the full data (takes much longer)
SAMPLE = False

## Ratings

This section performs rating normalization (technically standardization) to all ratings

In [ ]:
# Load ratings and users data
file_name = f"data/01_cleaned/{'sample/' if SAMPLE else ''}ratings.csv"
ratings = pd.read_csv(file_name, header=0)
file_name = f"data/01_cleaned/{'sample/' if SAMPLE else ''}users.csv"
users = pd.read_csv(file_name, header=0)

In [ ]:
# Index on the "user_id" and "rating_id" columns
ratings = ratings.set_index(["user_id", "rating_id"])
# Index on the "user_id" column for the users data
users = users.set_index("user_id")

In [ ]:
# Find the number of ratings for each user_id in the ratings data frame
user_ratings = ratings.groupby("user_id").agg(
    num_ratings = pd.NamedAgg(column="stars", aggfunc="count"),
    mean_rating = pd.NamedAgg(column="stars", aggfunc="mean"),
    std_rating = pd.NamedAgg(column="stars", aggfunc="std"),
)
user_ratings.head()

In [ ]:
# Add a "num_ratings" to the users data frame by joining the users and user_ratings data frames
users = users.join(user_ratings, on="user_id", how="left")
# Set null values to 0
users["num_ratings"].fillna(0, inplace=True)
users.head()

In [ ]:
# Add a "above_cutoff" column to the users data frame
# This column is True if the user has more than "cutoff" ratings in the ratings data frame
cutoff = 5 # TODO justify this
users["above_cutoff"] = users["num_ratings"] > cutoff
users.head()

In [ ]:
ratings["std_rating"] = np.NaN
cutoff_users = users[users["above_cutoff"]]
ratings.loc[cutoff_users.index, "std_rating"] = (
    ratings.loc[cutoff_users.index, "stars"] - cutoff_users["mean_rating"]
) / cutoff_users["std_rating"]

In [ ]:
ratings.loc[cutoff_users.index].head(20)

In [ ]:
other_users = users[~users["above_cutoff"]]
other_mean = ratings.loc[other_users.index, "stars"].mean()
other_std = ratings.loc[other_users.index, "stars"].std()
ratings.loc[other_users.index, "std_rating"] = (
    ratings.loc[other_users.index, "stars"] - other_mean
) / other_std

In [ ]:
ratings.loc[other_users.index].head(20)

In [ ]:
min_std = ratings["std_rating"].min()
max_std = ratings["std_rating"].max()
ratings["norm_std_rating"] = (((ratings["std_rating"] - min_std) / (max_std - min_std)) * 4) + 1
ratings.head(20)

In [ ]:
reindexed_ratings = ratings.reset_index()
reindexed_ratings.head()

In [ ]:
file_name = f"data/02_postprocessing/{'sample/' if SAMPLE else ''}ratings.csv"
with open(file_name, "w") as f:
    reindexed_ratings.to_csv(f, index=False, header=True)

## Businesses

### Categories >> Type, Keywords

In [ ]:
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}businesses.json") as f:
    businesses = [json.loads(line) for line in f]

In [ ]:
# Count most frequently category occurrences in businesses.json
category_counts = {}
for business in businesses:
    categories_str = business.get("categories", '')
    if categories_str:
        categories = categories_str.split(', ')
        for category in categories:
            category_counts[category] = category_counts.get(category, 0) + 1

# Find the most frequent categories
category_counts = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)
category_counts[:1000]

# Create a list of the most frequent categories
top_categories = [category[0] for category in category_counts[:1000]]

# print the top 500 most frequent categories
print(top_categories)

# Keep only the businesses that have at least one of the top 5 categories using a dictionary and create a new attribute "type" for each business that states which of the top 5 categories it belongs to
businesses_dict = {}
for business in businesses:
    categories_str = business.get("categories", '')
    if categories_str:
        categories = categories_str.split(', ')
        for category in categories:
            if category in top_categories:
                business["type"] = category
                businesses_dict[business["business_id"]] = business
                break
        else:
            # If no matching category is found, assign the business to the "Other" category
            business["type"] = "Other"
            businesses_dict[business["business_id"]] = business

# Define category grouping based on similarity
category_groups = {
    'Restaurants': ['Restaurants','Food', 'Breakfast & Brunch', 'Fast Food', 'Burgers', 'Pizza', 'Sandwiches', 'American (Traditional)', 'American (New)', 'Mexican', 'Chicken Wings', 'Salad', 'Chinese', 'Cafes', 'Sushi Bars', 'Barbeque', 'Southern', 'Japanese', 'Steakhouses', 'Juice Bars & Smoothies', 'Asian Fusion', 'Diners', 'Tex-Mex', 'Thai', 'Mediterranean', 'Indian', 'Vietnamese', 'Cajun/Creole', 'Latin American', 'Cuban', 'Puerto Rican', 'Caribbean', 'Japanese', 'Spanish', 'Korean', 'French', 'Halal', 'Mongolian', 'Canadian (New)', 'Filipino', 'Greek', 'Brazilian', 'Argentine', 'Hawaiian', 'Afghan', 'Indonesian', 'German', 'Ramen', 'Poke', 'Hot Pot', 'German', 'Pakistani', 'Uzbek', 'Persian/Iranian', 'Russian', 'Burmese', 'New Mexican Cuisine'],
    'Shopping': ['Shopping', 'Fashion', 'Specialty Food', 'Bakeries', 'Grocery', 'Flowers & Gifts', 'Furniture Stores', 'Jewelry', 'Shoe Stores', 'Accessories', 'Vintage & Consignment', 'Sporting Goods', 'Beer, Wine & Spirits', 'Department Stores', 'Bookstores', 'Electronics', 'Drugstores', 'Music & DVDs', 'Toy Stores', 'Tattoo', 'Eyewear & Opticians', 'Optometrists', 'Comic Books', 'Antiques', 'Gift Shops', 'Mobile Phones', 'Discount Store', 'Hardware Stores', 'Pet Stores', 'Appliances & Repair', 'Appliances', 'Vape Shops', 'Outlet Stores', 'Home & Garden', 'Building Supplies', 'Art Supplies', 'Thrift Stores', 'Hobby Shops', 'Musical Instruments & Teachers', 'Mattresses', 'Bike Repair/Maintenance', 'Books, Mags, Music & Video', 'Computers', 'Mobile Phone Accessories', 'Cosmetics & Beauty Supply', 'Eyelash Service', 'Hair Extensions', 'Hair Stylists', 'Waxing', 'Cosmetic Surgeons', 'Makeup Artists', 'Hair Loss Centers', 'Medical Spas', 'Nail Technicians', 'Hair Salons', 'Nail Salons', 'Massage', 'Day Spas', 'Barbers', 'Massage Therapy', 'Tattoo Removal', 'Piercing', 'Laser Hair Removal', 'Eyebrow Services', 'Permanent Makeup', 'Acne Treatment', 'Weight Loss Centers', 'Medical Supplies', 'Optometrists', 'Health Markets', 'Nutritionists', 'Physical Therapy', 'Dermatologists', 'Chiropractors', 'Hospitals', 'Acupuncture', 'Cryotherapy', 'Medical Centers', 'Sports Medicine', 'Alternative Medicine', 'Prenatal/Perinatal Care', 'Ophthalmologists', 'Internal Medicine', 'Allergists', 'Audiologist', 'Ear Nose & Throat', 'Allergists'],
    'Home Services': ['Home Services', 'Local Services', 'Real Estate', 'Contractors', 'Apartments', 'Home Decor', 'Movers', 'Mattresses', 'Appliances & Repair', 'Appliances', 'Home & Garden', 'Building Supplies', 'Security Systems', 'Waterproofing', 'Insulation Installation', 'Pest Control', 'Heating & Air Conditioning/HVAC', 'Plumbing', 'Carpet Cleaning', 'Air Duct Cleaning', 'Home Inspectors', 'Electricians', 'Solar Installation', 'Solar Panel Cleaning', 'Windows Installation', 'Glass & Mirrors', 'Window Washing', 'Pressure Washers', 'Roofing', 'Gutter Services', 'Siding', 'Carpenters', 'Masonry/Concrete', 'Fireplace Services', 'Carpeting', 'Home Automation', 'Handyman', 'Painters', 'Landscape Architects', 'Irrigation', 'Fences & Gates', 'Pool & Hot Tub Service', 'Pool Cleaners', 'Tree Services', 'Septic Services', 'Water Purification Services', 'Oil Change Stations', 'Excavation Services', 'Snow Removal'],
    'Beauty & Spas': [ 'Beauty & Spas', 'Hair Salons', 'Nail Salons', 'Hair Removal', 'Skin Care', 'Day Spas', 'Barbers', 'Massage', 'Waxing', 'Massage Therapy', 'Tattoo Removal', 'Piercing', 'Laser Hair Removal', 'Eyebrow Services', 'Permanent Makeup', 'Acne Treatment', 'Weight Loss Centers', 'Health Markets', 'Nutritionists', 'Physical Therapy', 'Dermatologists', 'Chiropractors', 'Medical Spas', 'Cosmetic Surgeons', 'Makeup Artists', 'Hair Loss Centers', 'Medical Supplies', 'Optometrists', 'Alternative Medicine', 'Prenatal/Perinatal Care', 'Ophthalmologists', 'Internal Medicine', 'Allergists', 'Audiologist', 'Ear Nose & Throat', 'Allergists'],
    'Nightlife': ['Nightlife', 'Bars', 'Sports Bars', 'Pubs', 'Cocktail Bars', 'Beer Gardens', 'Wine Bars', 'Karaoke', 'Breweries', 'Dive Bars', 'Wine Tasting Room', 'Jazz & Blues', 'Tiki Bars', 'Bartenders', 'Distilleries'],
    'Other': []
}


# Initialize grouped category counts
grouped_category_counts = {}

# Iterate through businesses and update the grouped categories
for business in businesses:
    categories_str = business.get("categories", '')
    if categories_str:
        categories = categories_str.split(', ')
        for category in categories:
            for group, group_categories in category_groups.items():
                if category in group_categories:
                        grouped_category_counts[group] = grouped_category_counts.get(group, 0) + 1
                        business["type"] = group
                        businesses_dict[business["business_id"]] = business
                        break
            else:
                # If no matching category is found, assign the business to the "Other" category
                grouped_category_counts["Other"] = grouped_category_counts.get("Other", 0) + 1
                business["type"] = "Other"
                businesses_dict[business["business_id"]] = business
                        

# Find the most frequent grouped categories
grouped_category_counts = sorted(grouped_category_counts.items(), key=lambda x: x[1], reverse=True)
grouped_category_counts[:6]

# Create a list of the most frequent grouped categories
top_grouped_categories = [category[0] for category in grouped_category_counts[:6]]

# print the top 5 most frequent grouped categories and their counts
print(grouped_category_counts[:6])


In [ ]:

plt.pie([category[1] for category in category_counts])
plt.title("Original Categories")
plt.show()


In [ ]:
# Define data for the pie chart
labels = top_grouped_categories
sizes = [count[1] for count in grouped_category_counts[:6]]

# Create a pie chart
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.
plt.title('Grouped Categories Distribution')

# Show the pie chart
plt.show()

# print out the grouped categories and their counts 
grouped_category_counts

In [ ]:
# Clean up the data to save for query processing

# Remove the "categories" key from test business in business_dict
for business in businesses_dict.values():
    business.pop("categories", None)
    
# Make it into a data frame
business_df = pd.DataFrame(businesses_dict.values())
business_df["type"] = business_df["type"].str.lower() # Make type lowercase

In [ ]:
file_name = f"data/02_postprocessing/{'sample/' if SAMPLE else ''}businesses.csv"
with open(file_name, "w") as f:
    business_df.to_csv(f, index=False, header=True)

### Keyword Extraction

In [ ]:
# read business JSON file
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}businesses.json") as f:
    businesses = [json.loads(line) for line in f]

In [ ]:
stop_words = set(stopwords.words("english"))
custom_stop_words = [",", "&", "-", "(", ")", ".", "'", "!", "?", ":", ";", "[", "]", "/"]

In [ ]:
# Extract keywords from each business entry for category and name attributes using dictionary
business_keywords = {}
for business in businesses:
    business_keywords[business["business_id"]] = {"category": [], "name": []}
    for word in word_tokenize(business["categories"]):
        if word not in stop_words and word not in custom_stop_words:
            business_keywords[business["business_id"]]["category"].append(word)
    for word in word_tokenize(business["name"]):
        if word not in stop_words and word not in custom_stop_words:
            business_keywords[business["business_id"]]["name"].append(word)

In [ ]:
# read review JSON file
with open(f"data/01_cleaned/{'sample/' if SAMPLE else ''}reviews.json") as f:
    reviews = [json.loads(line) for line in f]

In [ ]:
# Extract keywords from each review entry for text attribute using dictionary
review_keywords = {}
for review in reviews:
    review_keywords[review["review_id"]] = []
    for word in word_tokenize(review["text"]):
        if word not in stop_words and word not in custom_stop_words:
            review_keywords[review["review_id"]].append(word)

In [ ]:
# Create a dictionary of business_id to a list of review_ids
business_review_dict = {}
for review in reviews:
    if review["business_id"] not in business_review_dict.keys():
        business_review_dict[review["business_id"]] = [review["review_id"]]
    else:
        business_review_dict[review["business_id"]].append(review["review_id"])

In [ ]:
# combine keywords from all reviews, category, and name for each business
# some businesses do not have any reviews, so we only combine category and name keywords for those businesses using dictionary
business_keywords_combined = {}
for business in businesses:
    business_keywords_combined[business["business_id"]] = []
    if business["business_id"] in business_review_dict.keys():
        for review_id in business_review_dict[business["business_id"]]:
            business_keywords_combined[business["business_id"]] += review_keywords[review_id]
    business_keywords_combined[business["business_id"]] += business_keywords[business["business_id"]]["category"]
    business_keywords_combined[business["business_id"]] += business_keywords[business["business_id"]]["name"]

In [ ]:
# print the keywords for each business with their name
for business in businesses:
    print(business["name"] + ": ")
    print(business_keywords_combined[business["business_id"]])
    print("\n")